In [15]:
import cobra
from cobra import Reaction, Metabolite, Model
import pandas as pd
import numpy

In [16]:
#loading new metabolite data
current_model = cobra.io.read_sbml_model("../results/reconstructions/v5_with_all_annotations.xml")

#loading new metabolite data
meta_data = pd.read_csv("../data/metabolite_ids_Anderson2014.csv")
print(meta_data)

                                             met_name    met_id    source
0                                   L-(+) lactic acid  cpd00159  Anderson
1                                       glycolic acid  cpd00139  Anderson
2                                           L-alanine  cpd00035  Anderson
3                      3-hydroxypropanoic acid (NIST)  cpd00745  Anderson
4                                           L-proline  cpd00129  Anderson
5                       dehydroalanine from cysteine*  cpd01495  Anderson
6   3-hydroxy-3-methylbutanoic acid (3-hydroxyisov...       NaN  Anderson
7                                            L-valine  cpd00156  Anderson
8                                                urea  cpd00073  Anderson
9                                        benzoic acid  cpd00153  Anderson
10                                           L-serine  cpd00054  Anderson
11                                    phosphoric acid  cpd00009  Anderson
12                                    

In [41]:

#checking for metabolite in current model (cpdXXXXX_e or _c)

in_mod = []
out_mod = []
met_id = []

original_met_id = meta_data['met_id'].tolist()
for cpd_id in original_met_id:
    if "cpd" in str(cpd_id):
        met_id.append(cpd_id)
        
for cpd_id in met_id:
    try:
        current_model.metabolites.get_by_id(str(cpd_id)+"_c" or "_e")
        in_mod.append(cpd_id)
    except:
         out_mod.append(cpd_id)
print(out_mod)          

[]


In [43]:
#checking if metabolite has exchange reaction
RX_in_mod = []
RX_out_mod = []

for cpd_id in in_mod:
    try:
        current_model.reactions.get_by_id("EX_"+str(cpd_id))
        RX_in_mod.append(cpd_id)
    except:
        RX_out_mod.append(cpd_id)


In [44]:
#UNNECESSARY all existing metabolites already have exchange rxns
#create exchange reaction of existing metabolites w bounds -1000,1000
for cpd_id in RX_out_mod:
    Ex_rxn = cobra.Reaction("EX_" + str(cpd_id) + "_c")
    met_obj = current_model.metabolites.get_by_id(str(cpd_id)+"_c" or "_e")
    Ex_rxn.add_metabolites({met_obj.copy():-1})

    Ex_rxn.id = "EX_"+str(cpd_id)
    Ex_rxn.lower_bound = -1000
    Ex_rxn.upper_bound = 1000
    current_model.add_reaction(Ex_rxn)
    
    

In [46]:
#loading universal model for meatbolites not found in current model
universal = cobra.io.load_json_model("../data/modelseed_data/seed_universal.json")

In [47]:
#checking if metabolites not in model are in universal
in_uni = []
out_uni = []

for cpd_id in out_mod:
    try:
        universal.metabolites.get_by_id(str(cpd_id)+"_c" or "_e" or "")
        in_uni.append(cpd_id)
    except:
         out_uni.append(cpd_id)

print(out_uni)

[]


In [49]:
#make copy of metabolite from universal
#create exchange reaction for universal metabolites w bounds -1000,1000
for cpd_id in in_uni:
    met_obj = universal.metabolites.get_by_id(str(cpd_id)+"_c" or "_e")
    Ex_rxn = cobra.Reaction("EX_" + str(cpd_id))
    Ex_rxn.add_metabolites({met_obj.copy():-1})
    
    Ex_rxn.id = "EX_"+str(cpd_id)
    Ex_rxn.lower_bound = -1000
    Ex_rxn.upper_bound = 1000
    current_model.add_reaction(Ex_rxn)
    


In [50]:
#save model with new exchange reactions
cobra.io.write_sbml_model(current_model, "v6_with_all_annotations.xml", use_fbc_package=True)